# Neural Networks Example

This notebook provides a brief (but useful!) intro to using neural networks in Python.

For this, we will use the `sklearn` package and use it to classify our breast cancer dataset.

In [ ]:
# Imports
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd
import os

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve, roc_auc_score
%matplotlib inline

# Size, passed to `plt.figure(figsize=FIGSIZE)`
FIGSIZE = (10, 6)

# DPI for saving the figure
FIGDPI = 200

# Number of points to use for toy distributions
NPTS = 200

# Pull matlab colormap
cmap = plt.rcParams['axes.prop_cycle'].by_key()['color']

# Random Seed
np.random.seed(100)

## Loading the Data

As always, we load the data into Python using the `pandas` package.

In [ ]:
# Set up path to the sample data
DATAPATH = os.path.join('data.csv')
df = pd.read_csv(DATAPATH)

# Display sample of the data
df.head()

## Preparing Data

First, we pull out the relevant features -- for this network, we are interested in the texture mean and radius mean of the dataset. 
We pull these out into `numpy` arrays, and then stack them so that we have an $N \times d$ matrix, where $N$ is the number of samples and $d=2$ is our dimensionality (i.e. the number of "input" units).

In [ ]:
# =================================================
# Use this code to classify using only two features
# =================================================

X1 = np.array(df.texture_mean)
X2 = np.array(df.radius_mean)
X = np.transpose(np.vstack((X1, X2)))

# =================================================
# Use this code to classify using all features
# =================================================
# X = np.array(df.iloc[:,2:])

Next, we have to grab a label vector of $N\times1$ size, indicating the labels for each sample. `sklearn` provides a nice function for this, `preprocessing.LabelEncoder()`, which you can use to transform the `pandas` dataframe into something usable.

In [ ]:
# Convert text labels into numbers
le = preprocessing.LabelEncoder()
le.fit(['B', 'M'])
y = le.transform(df.diagnosis)

Now that we have our data matrix $\mathbf{X}$ and our label vector $\mathbf{y}$, we have to separate them into training and testing sets. Again, `sklearn` provides a convenience function for this, `train_test_split()`, which allows you to specify a percentage of data to use for testing. Here we set `test_size=.3`, meaning that we are using 30% of the data for testing and 70% of the data for training.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

Finally, we need to scale the data. This helps the neural network in training by providing a nice set of inputs to learn from, and can help with convergence. The `StandardScalar()` object calculates the scaled feature $i$ vector as:

$$ \mathbf{x}_{i}^{\prime} = \frac{\mathbf{x}_{i} - \mu_{i}}{\sigma_{i}} $$

Where $\mathbf{x}_{i}$ is the $i$-th feature, and $\mu_{i}$ and $\sigma_{i}$ are the mean and standard deviation for that feature. The result is a vector which has $\mu = 0$ and $\sigma = 1$. Note that each feature is scaled independently from the others. 

Also, note that we need to calculate $\mu$ and $\sigma$ on the **TRAINING** data, and then apply that scaling to both the training and the testing data. This is because we aren't supposed to know about the testing data yet -- thus, the mean and standard deviation shouldn't include these values.

In [ ]:
# Scale the data
scaler = StandardScaler()
scaler.fit(X_train) 
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Training the Neural Network

At last, we train! First, we create the classification "object" for a multilayer perceptron. You can look [here at the documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html) for a full list of the input parameters that you can specify.

Here, we specify the following:

- `solver`: "sgd" for Stochastic Gradient Descent
- `alpha`: This is the L2-Regularization term -- basically it helps smooth out the gradients during training
- `max_iter`: Tells the network to stop training at a certain number of iterations (otherwise it would keep going until the network "converges", which may take a really long time)
- `hidden_layer_sizes`: Specifies the "structure" of the network -- how many hidden layers, and how many neurons per layer?
- `random_state`: Allows us to reproduce the training if we run the algorithm again -- remember, the process is not deterministic, so you may get different results if you run multiple times!

In [ ]:
# try hidden_layer_sizes = (4), (4, 4), (4, 4, 2)

clf = MLPClassifier(solver='sgd', alpha=1e-5, max_iter=2000,
                    hidden_layer_sizes=(4), random_state=1)


Now that we have our classifier object, we can simply fit the model to the data (another way of saying "training"). This command will also print out a summary of all the options of the classifier.

In [ ]:
clf.fit(X_train, y_train)

Let's see what the performace of the fit classifier is on the training data. This gives us an idea if we're doing well at all.

In [ ]:
scores = cross_val_score(clf, X_train, y_train, cv=3)
print('Accuracy: {:0.2f}% (+/- {:0.2f})'.format(scores.mean(), scores.std() * 2))

## Testing the Network

Now all that's left to do is apply the trained classifier to our source data. Most classifiers in `sklearn` have a `predict` function, but in this case we are also interested in the probabilities of the testing data, so let's use `predict_proba()` instead.

In [ ]:
# Apply the fitted classifier to our data (overfitting!)
x_predict = clf.predict_proba(X_test)

How have we done? Let's look at the probabilities for some of the data (first ten elements), along with their ground truth labels. We'll also print out the predicted class, which is just which label had the higher probability.

In [ ]:
print('P1\tP2\tPc\tLabel\n')
for i in range(10):
    print('{:0.2f}\t{:0.2f}\t{:d}\t{:d}\n'.format(x_predict[i,0], x_predict[i,1], np.argmax(x_predict[i,:]), y_test[i]))

Not bad! Let's get a performance evaluation of this.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, x_predict[:,1])
auc = roc_auc_score(y_test, x_predict[:,1])

In [ ]:
# Create the plot
f, ax = plt.subplots(figsize=(6,6))

# Plot the data
plt.plot(fpr, tpr, color=cmap[0], label="ROC Curve")
plt.plot(np.linspace(0,1), np.linspace(0,1), color=cmap[1], label="0.5")

# Tweak the plot
ax.legend(frameon=True, loc='upper left')
ax.set(xlabel='False Positive Rate',
      ylabel='True Positive Rate',
      title='ROC Curve (AUC: {:.3f})'.format(auc))
ax.grid(linestyle=':')
plt.tight_layout()
plt.show()

## Visualize Results

It's always a good idea to visualize what you've done to verify that your classifier did what you expected it to do. So here, we'll plot out a region of space defined by the feature limits, then apply the classifier and look to see where our samples land.

In [ ]:
# Plot Contours
# step size in the mesh
h = .02  

cm = plt.cm.RdBu_r
cm_bright = ListedColormap(['#FF0000', '#0000FF'])
    
x_min, x_max = X_train[:, 0].min() - .5, X_train[:, 0].max() + .5
y_min, y_max = X_train[:, 1].min() - .5, X_train[:, 1].max() + .5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

if hasattr(clf, "decision_function"):
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
else:
    Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

f, ax = plt.subplots(figsize=FIGSIZE)
    
# Put the result into a color plot
Z = Z.reshape(xx.shape)
ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

# Plot also the training points
# ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
#            edgecolors='black', s=80, alpha=.8)
ax.scatter(X_train[y_train==0,0], X_train[y_train==0,1], edgecolors='black', s=80, alpha=.8, label='Benign')
ax.scatter(X_train[y_train==1,0], X_train[y_train==1,1], edgecolors='black', s=80, alpha=.8, label='Malignant')

# ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
#            edgecolors='black', s=35)

ax.legend(frameon=True, loc='upper left')
ax.set(xlabel='Texture Mean',
       ylabel='Radius Mean',
       xlim=(xx.min(), xx.max()),
       ylim=(yy.min(), yy.max()),
       title='Neural Network Classification Landscape')
ax.grid(linestyle=':')